# Overview: Implementing vaccination agent using Group Chat design

## Group chat design
a group of agents share a **common thread of messages**: they **all subscribe and publish to the same topic**. Each participant agent is **specialized for a particular task**, such as writer, illustrator, and editor in a collaborative writing task. You can also include an agent to represent a human user to help guide the agents when needed.

### sequential nature, controlled by Group Chat Mansger agent: 
participants take turn to publish a message, only one agent is working at a time. **Group Chat Manager** agent is responsible for **selecting the next agent** to speak **upon receiving a message**.


### flexibility: 
It is also possible to nest group chats into a hierarchy with **each participant a recursive group chat**.

## Vaccination agent

1. Planner: The planning agent plans and delegates the tasks to specific agents

2. Vaccine records retrieval agent: Retrieves user vaccination history and health data.
    a. data will include user profile info and vaccination history

3. Vaccination recommender agent: Provides personalised recommendations based on the user’s vaccine records and profile.
    a. combine user health data and external knowledge sources (e.g., official guidelines) to provide accurate recommendations.
    b. handle general vaccine FAQs and side effects (option to also have a separate “knowledge agent.”)
    
4. Appointment checker agent: Checks for available slots or slot already booked by user and assists in booking/ cancellation/ vaccination appointments.
    a. Queries slots availability.
    b. Responsible for finalizing, modifying, or canceling appointments.
    c. Could require user confirmation or additional info (e.g., location preferences, times, contact info).

### workflow
1. assume that the user query is vaccination-related, Manager will first direct the message to planner, then the planner will return the plan in the format of "<No.>. <which_agent>: <task> to manager
2.  then the manager will follow the plan to pass the chat history and the task plan, specific task to do to next agent, then the agent will perform task (call external function if needed), return the result to the manager, with his task cross out in the task list , together with the remaining chat history,
3. this goes so and so for until all task has been done, and the manager return the final result (and task completion message) to the user 

# Setting up the Agent

In [1]:
import asyncio
from typing import List

import nest_asyncio

# testing openai connection
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import (
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
)
from autogen_core._type_prefix_subscription import TypePrefixSubscription
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from pydantic import BaseModel

# from rich.console import Console
from rich.markdown import Markdown

In [6]:
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Get values from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_AD_TOKEN_SCOPE = os.getenv("AZURE_AD_TOKEN_SCOPE")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_CHATGPT_MODEL = os.getenv("AZURE_OPENAI_CHATGPT_MODEL")
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")

In [7]:
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, AZURE_AD_TOKEN_SCOPE)

# model_client = AzureOpenAI(
#     api_version=AZURE_OPENAI_API_VERSION,
#     azure_endpoint=AZURE_OPENAI_ENDPOINT,
#     azure_ad_token_provider=token_provider,
# )

# response = model_client.chat.completions.create(
#     model=AZURE_OPENAI_MODEL,
#     messages=[{"role": "user", "content": "Hello, how can I assist you?"}],
#     max_tokens=50,
#     # test
# )

In [8]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider,
)


async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
# if __name__ == "__main__":
#     asyncio.run(main())

# Call the wrapper function in your notebook cell
nest_asyncio.apply()  # patch the loop
asyncio.run(main())

---------- user ----------
What is the weather in New York?


---------- weather_agent ----------
[FunctionCall(id='call_EKSdHemR9YKsCAxje9QRob5i', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_EKSdHemR9YKsCAxje9QRob5i', is_error=False)]
---------- weather_agent ----------
The weather in New York is currently 73 degrees and sunny.


In [22]:
class GroupChatMessage(BaseModel):
    """Message to sent back to the agent"""

    body: UserMessage


class RequestToSpeak(BaseModel):
    body: UserMessage

# 0.) Utility function

In [23]:
# List all subscriptions in a runtime


def list_all_sub(runtime):
    subscriptions = runtime._subscription_manager._subscriptions
    print("x" * 100)
    print("Current Subscriptions:")

    for sub in subscriptions:
        print(type(sub), end=": ")
        if isinstance(sub, TypeSubscription):
            print(
                "TypeSubscription, topic type:",
                sub._topic_type,
                "agent type:",
                sub._agent_type,
            )
        elif isinstance(sub, TypePrefixSubscription):
            print(
                "TypePrefixSubscription, topic_type_prefix:",
                sub.topic_type_prefix,
                "agent type:",
                sub.agent_type,
            )
        else:
            print("another subscription type:", type(sub))
    print("x" * 100)

# 1.) User Agent
- uses console input to get the user’s input. In a real-world scenario, you can replace this by communicating with a frontend, and subscribe to responses from the frontend.



In [24]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(
        self, message: GroupChatMessage, ctx: MessageContext
    ) -> None:
        # When integrating with a frontend, this is where group chat message would be sent to the frontend.
        # print(f"\n{'-'*80}\nUser receive GroupChatMessage (do nothing for now):", message.body.content)
        self._chat_history.extend(
            [
                UserMessage(
                    content=f"Transferred to {message.body.source}", source="system"
                ),
                message.body,
            ]
        )
        pass

    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(f"\n{'-'*80}\nUser received 'RequestToSpeak' message")
        print("current chat history:")
        for message in self._chat_history:
            print(f"{message.source}: {message.content}")

        user_input = input("Enter your message, type 'APPROVE' to conclude the task: ")
        print(Markdown(f"User Response: \n{user_input}"))
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=user_input, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )

# 2.) Base Group Chat Agent
- used as the base class for all AI agents in the group chat.

In [25]:
class BaseGroupChatAgent(RoutedAgent):
    """A group chat participant using an LLM."""

    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
        system_message: str,
    ) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type
        self._model_client = model_client
        self._system_message = SystemMessage(content=system_message)
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(
        self, message: GroupChatMessage, ctx: MessageContext
    ) -> None:
        """Append the message to the chat history"""
        # print(f"\n{'-'*80}\n{self.id.type}: BaseGroupChatAgent handle message, add to _chat_history", flush=True)
        self._chat_history.extend(
            [
                UserMessage(
                    content=f"Transferred to {message.body.source}", source="system"
                ),
                message.body,
            ]
        )

    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(
            f"\n{'-'*80}\n{self.id.type}: BaseGroupChatAgent handle RequestToSpeak",
            flush=True,
        )
        print(f"RequestToSpeak message: {message.body.content}", flush=True)
        # Console().print(Markdown(f"### {self.id.type}: "))

        self._chat_history.append(
            UserMessage(
                content=f"Transferred to {self.id.type}, adopt the persona immediately.",
                source="system",
            )
        )

        # Create a completion
        completion = await self._model_client.create(
            [self._system_message] + self._chat_history
        )
        assert isinstance(completion.content, str)
        self._chat_history.append(
            AssistantMessage(content=completion.content, source=self.id.type)
        )
        # Console().print(Markdown(completion.content))
        print(
            "Completion content under BaseGroupChatAgent,",
            completion.content,
            flush=True,
        )

        # publish the completion
        await self.publish_message(
            GroupChatMessage(
                body=UserMessage(content=completion.content, source=self.id.type)
            ),
            topic_id=DefaultTopicId(type=self._group_chat_topic_type),
        )

# 3.) Agents: Planner, Record, Recommendation, Booking slot Agents

In [26]:
# Defining the Planner Agent


class PlannerAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Planner. Break down tasks and assign them to the appropriate agents.",
        )

    # assume fixed plan for planner
    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(f"\n{'-'*80}\nPlannerAgent - handle_request_to_speak")

        plan = "1. VaccineRecords: Retrieve user vaccination history.\n2. VaccinationRecommender: Provide vaccine recommendations.\n3. AppointmentChecker: Check appointment availability."
        print(f"Return dummp plan:\n{plan}")
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=plan, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )


# Defining the Vaccine Records Retrieval Agent
class VaccineRecordsAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Vaccine Records Retrieval Agent. You fetch user vaccination history and health data.",
        )

    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(f"\n{'-'*80}\nVaccineRecordsAgent - handle_request_to_speak")
        dummy_data = "User's vaccine history: COVID-19 (Pfizer, 2023), Flu Shot (2022), Hepatitis B (2020)."
        print(f"Retrived and return dummy history:\n{dummy_data}")
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=dummy_data, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )


# Defining the Vaccination Recommender Agent
class VaccinationRecommenderAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Vaccination Recommender Agent. You provide personalized vaccine recommendations.",
        )

    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(f"\n{'-'*80}\nVaccinationRecommenderAgent - handle_request_to_speak")
        dummy_recommendation = "Recommended vaccines: Flu Shot (2024), HPV (if applicable), COVID-19 booster."
        print(f"Return dummp recommendation:\n{dummy_recommendation}")
        await self.publish_message(
            GroupChatMessage(
                body=UserMessage(content=dummy_recommendation, source=self.id.type)
            ),
            DefaultTopicId(type=self._group_chat_topic_type),
        )


# Defining the Appointment Checker Agent
class AppointmentCheckerAgent(BaseGroupChatAgent):
    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
    ) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are an Appointment Checker Agent. You check and manage vaccination appointments.",
        )

    @message_handler
    async def handle_request_to_speak(
        self, message: RequestToSpeak, ctx: MessageContext
    ) -> None:
        print(f"\n{'-'*80}\nAppointmentCheckerAgent - handle_request_to_speak")
        dummy_appointment = "Next available appointment: June 15, 2024, at 10:00 AM, City Health Clinic."
        print("Return dummy available appointment")
        await self.publish_message(
            GroupChatMessage(
                body=UserMessage(content=dummy_appointment, source=self.id.type)
            ),
            DefaultTopicId(type=self._group_chat_topic_type),
        )

# 4.) Group Chat Manager
- manages the group chat and selects the next agent to speak using an LLM.
- checks if the editor has approved the draft by looking for the `APPORVED` keyword in the message. If the editor has approved the draft, the group chat manager stops selecting the next speaker, and the group chat ends.

In [27]:
# Defining the Group Chat Manager Agent


class GroupChatManager(BaseGroupChatAgent):
    def __init__(
        self,
        participant_topic_types: list[str],
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
        participant_descriptions: list[str],
    ) -> None:
        super().__init__(
            description="Group Chat Manager.",
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="You are a Group Chat Manager. Manage the sequence of agent interactions.",
        )

        self._participant_topic_types = participant_topic_types
        self._participant_descriptions = participant_descriptions
        self._previous_participant_topic_type = None

    @message_handler
    async def handle_message(
        self, message: GroupChatMessage, ctx: MessageContext
    ) -> None:
        self._chat_history.append(message.body)

        # terminating condition
        if (
            message.body.source == "User"
            and message.body.content.lower().strip().endswith("approve")
        ):
            print("User approved, session terminated")
            return

        # select the next topic type based on the _chat_history, _participant_topic_types, and _participant_descriptions
        history = "\n".join(
            [f"{msg.source}: {msg.content}" for msg in self._chat_history]
        )
        roles = "\n".join(
            [
                f"{topic_type}: {description}"
                for topic_type, description in zip(
                    self._participant_topic_types, self._participant_descriptions
                )
                if topic_type != self._previous_participant_topic_type
            ]
        )
        system_message = SystemMessage(
            content=f"""Select the next role from {self._participant_topic_types} to act based on the following history:\n{history}\n\nRoles:\n{roles}"""
        )

        completion = await self._model_client.create(
            [system_message], cancellation_token=ctx.cancellation_token
        )
        assert isinstance(completion.content, str)
        selected_topic_type = next(
            (
                topic
                for topic in self._participant_topic_types
                if topic.lower() in completion.content.lower()
            ),
            None,
        )

        # print message
        print(
            f"\n{'-'*80}\n{self.id.type} receive GroupChatMessage:\n{message.body.content}"
        )
        print(f"- Chat history: {self._chat_history}", flush=True)
        print(
            f"- Completion message under GroupChatManager: {completion.content}",
            flush=True,
        )
        print(f"- Selected topic type: {selected_topic_type}", flush=True)

        # publish the selected topic type
        if selected_topic_type:
            self._previous_participant_topic_type = selected_topic_type
            await self.publish_message(
                RequestToSpeak(
                    body=UserMessage(content=selected_topic_type, source=self.id.type)
                ),
                DefaultTopicId(type=selected_topic_type),
            )
        else:
            raise ValueError(f"Invalid role selected: {completion.content}")

# Running & starting the Group Chat 

In [28]:
# Define agent topic types
manager_topic_type = "GroupChatManager"
planner_topic_type = "Planner"
vaccine_records_topic_type = "VaccineRecords"
recommender_topic_type = "VaccinationRecommender"
appointment_topic_type = "AppointmentChecker"
user_topic_type = "User"
group_chat_topic_type = "group_chat"


# Agent descriptions
planner_description = (
    "Planner agent responsible for breaking down tasks and delegating."
)
vaccine_records_description = (
    "Agent for retrieving user vaccination records and health data."
)
recommender_description = (
    "Agent for providing vaccine recommendations based on user data."
)
appointment_description = "Agent for checking and managing vaccination appointments."
user_description = "User interacting with the system."


async def main():
    runtime = SingleThreadedAgentRuntime()

    """
    1. Registering the Planner Agent, 
    register returns AgentType, i.e. AgentType(type='Planner')
    """
    planner_agent_type = await PlannerAgent.register(
        runtime,
        planner_topic_type,
        lambda: PlannerAgent(
            description=planner_description,
            group_chat_topic_type=group_chat_topic_type,  # broadcast message to this topic under handle_request_to_speak
            model_client=model_client,
        ),
    )

    await runtime.add_subscription(
        TypeSubscription(
            topic_type=planner_topic_type, agent_type=planner_agent_type.type
        )
    )  # Takes in topic_type and string agent_type, both are of string type
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=planner_agent_type.type
        )
    )

    """
    2. Registering the Vaccine Records Retrieval Agent, 
    AgentType(type='VaccineRecords')
    """
    vaccine_records_agent_type = await VaccineRecordsAgent.register(
        runtime,
        vaccine_records_topic_type,
        lambda: VaccineRecordsAgent(
            description=vaccine_records_description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
        ),
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=vaccine_records_topic_type,
            agent_type=vaccine_records_agent_type.type,
        )
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=vaccine_records_agent_type.type
        )
    )

    """3. Registering the Vaccination Recommender Agent"""
    recommender_agent_type = await VaccinationRecommenderAgent.register(
        runtime,
        recommender_topic_type,
        lambda: VaccinationRecommenderAgent(
            description=recommender_description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
        ),
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=recommender_topic_type, agent_type=recommender_agent_type.type
        )
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=recommender_agent_type.type
        )
    )

    """ 4. Registering the Appointment Checker Agent"""
    appointment_agent_type = await AppointmentCheckerAgent.register(
        runtime,
        appointment_topic_type,
        lambda: AppointmentCheckerAgent(
            description=appointment_description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
        ),
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=appointment_topic_type, agent_type=appointment_agent_type.type
        )
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=appointment_agent_type.type
        )
    )

    """ 5. Registering the Appointment Checker Agent """
    user_agent_type = await UserAgent.register(
        runtime,
        user_topic_type,
        lambda: UserAgent(
            description=user_description, group_chat_topic_type=group_chat_topic_type
        ),
    )
    await runtime.add_subscription(
        TypeSubscription(topic_type=user_topic_type, agent_type=user_agent_type.type)
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=user_agent_type.type
        )
    )

    # Registering the Group Chat Manager
    group_chat_manager_type = await GroupChatManager.register(
        runtime,
        manager_topic_type,
        lambda: GroupChatManager(
            participant_topic_types=[
                planner_topic_type,
                vaccine_records_topic_type,
                recommender_topic_type,
                appointment_topic_type,
                user_topic_type,
            ],
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            participant_descriptions=[
                planner_description,
                vaccine_records_description,
                recommender_description,
                appointment_description,
                user_description,
            ],
        ),
    )
    await runtime.add_subscription(
        TypeSubscription(
            topic_type=group_chat_topic_type, agent_type=group_chat_manager_type.type
        )
    )

    # Start runtime
    runtime.start()
    # session_id = str(uuid.uuid4())

    # subscribed_agents = await runtime._subscription_manager.get_subscribed_recipients(
    #     DefaultTopicId(type="VaccinationRecommender")
    # )

    print(
        "runtime starts, publish user message 'I need a vaccine recommendation and want to book an appointment.'"
    )
    await runtime.publish_message(
        GroupChatMessage(
            body=UserMessage(
                content="I need a vaccine recommendation and want to book an appointment.",
                source="User",
            )
        ),
        TopicId(type=group_chat_topic_type, source="default"),
    )

    await runtime.stop_when_idle()


nest_asyncio.apply()  # patch the loop
asyncio.run(main())

runtime starts, publish user message 'I need a vaccine recommendation and want to book an appointment.'



--------------------------------------------------------------------------------
GroupChatManager receive GroupChatMessage:
I need a vaccine recommendation and want to book an appointment.
- Chat history: [UserMessage(content='I need a vaccine recommendation and want to book an appointment.', source='User', type='UserMessage')]
- Completion message under GroupChatManager: VaccinationRecommender
- Selected topic type: VaccinationRecommender

--------------------------------------------------------------------------------
VaccinationRecommenderAgent - handle_request_to_speak
Return dummp recommendation:
Recommended vaccines: Flu Shot (2024), HPV (if applicable), COVID-19 booster.

--------------------------------------------------------------------------------
GroupChatManager receive GroupChatMessage:
Recommended vaccines: Flu Shot (2024), HPV (if applicable), COVID-19 booster.
- Chat history: [UserMessage(content='I need a vaccine recommendation and want to book an appointment.', sour

# Analysis:


1. the group chat design is suitable for non-deterministic tasks, like the main orchastrator, which handles very different workflow, but not a rather fixed workflow like vaccination record -> recommendation -> booking slot. Main cost is spent on selector function of the group manager. It takes in full chat history and available agents to speak, then output the selected agent.

2. the planner is not called first, unless strong prompt engineering is done. If we use planner, then we shall have a fixed sequential flow (no selector function needed)


# test on vaccination agent